# Tutorial: Azure Blob Storage Authentication & Audio Playback

**Purpose:** Test Azure blob storage connection and verify audio files are accessible

**What this notebook does:**
1. Load environment variables (Azure credentials)
2. Inspect the VHP parquet dataset
3. Pick one row and identify its blob path
4. Authenticate with Azure blob storage
5. Download and play the audio file

**Use this to debug:**
- Azure authentication issues
- Blob path mapping problems
- Audio file accessibility

---

## 1. Setup: Load Environment Variables and Imports

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd

# Load Azure credentials from .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path='../credentials/creds.env')

# Add scripts directory to path
sys.path.insert(0, str(Path.cwd().parent / "scripts"))

# Import Azure utilities
import azure_utils
import data_loader

print("✓ Imports loaded")
print("\nEnvironment variables check:")
print(f"  AZURE_STORAGE_CONTAINER: {os.getenv('AZURE_STORAGE_CONTAINER', 'NOT SET')}")
print(f"  AZURE_STORAGE_ACCOUNT: {os.getenv('AZURE_STORAGE_ACCOUNT', 'NOT SET')}")
print(f"  AZURE_AUTH: {os.getenv('AZURE_AUTH', 'NOT SET (defaults to managed_identity)')}")

## 2. Inspect Parquet Dataset

In [ ]:
# Load the VHP pre-2010 parquet file
parquet_path = "../data/raw/loc/veterans_history_project_resources_pre2010.parquet"
df = pd.read_parquet(parquet_path)

print(f"Parquet shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nKey columns for blob mapping:")
print(f"  - azure_blob_index: {'✓ Present' if 'azure_blob_index' in df.columns else '✗ Missing'}")
print(f"  - video_url: {df['video_url'].notna().sum()} items")
print(f"  - audio_url: {df['audio_url'].notna().sum()} items")
print(f"  - fulltext_file_str (transcripts): {df['fulltext_file_str'].notna().sum()} items")

## 3. Pick One Row with Media and Transcript

In [ ]:
# Filter for rows with both transcript and media
has_transcript = df['fulltext_file_str'].notna()
has_media = (df['video_url'].notna()) | (df['audio_url'].notna())
df_filtered = df[has_transcript & has_media]

print(f"Rows with both transcript and media: {len(df_filtered)}")

# Pick the first row from the sample we've been testing
# (parquet index 5047 -> azure_blob_index 6357)
test_parquet_idx = 5047
test_row = df.loc[test_parquet_idx]

print(f"\n{'='*70}")
print(f"Test Row Selected:")
print(f"{'='*70}")
print(f"Parquet index: {test_parquet_idx}")
print(f"Azure blob index: {int(test_row['azure_blob_index'])}")
print(f"Collection number: {test_row['collection_number']}")
print(f"Title: {test_row['title'][:80]}..." if len(str(test_row['title'])) > 80 else f"Title: {test_row['title']}")
print(f"Has video URL: {pd.notna(test_row['video_url'])}")
print(f"Has audio URL: {pd.notna(test_row['audio_url'])}")
print(f"Has transcript: {pd.notna(test_row['fulltext_file_str'])}")

## 4. Identify Blob Path for This Row

In [ ]:
# Use data_loader to construct blob path (same logic as inference scripts)
blob_prefix = "loc_vhp"
blob_path_candidates = data_loader.get_blob_path_for_row(
    test_row, 
    idx=test_parquet_idx, 
    blob_prefix=blob_prefix
)

print(f"\nBlob path candidates for this row:")
for i, path in enumerate(blob_path_candidates, 1):
    print(f"  {i}. {path}")

# These paths should match what commercial models successfully used
print(f"\n💡 Note: AWS Transcribe and Chirp2 successfully used these paths before")

## 5. Test Azure Authentication

In [ ]:
# Create blob service client
print("Creating Azure Blob Service client...\n")

try:
    blob_service = azure_utils.make_blob_service()
    print("✓ Blob service client created")
    
    # Test connection by listing containers
    print("\nTesting connection by listing containers...")
    containers = list(blob_service.list_containers())
    print(f"✓ Successfully authenticated! Found {len(containers)} container(s):")
    for container in containers:
        print(f"  - {container.name}")
    
    # Verify the expected container exists
    container_name = os.getenv('AZURE_STORAGE_CONTAINER')
    container_names = [c.name for c in containers]
    if container_name in container_names:
        print(f"\n✓ Target container '{container_name}' found")
    else:
        print(f"\n✗ WARNING: Target container '{container_name}' not found!")
        print(f"  Available containers: {container_names}")
    
except Exception as e:
    print(f"✗ Authentication failed: {e}")
    print(f"\nError type: {type(e).__name__}")
    import traceback
    print(f"\nFull traceback:\n{traceback.format_exc()}")

## 6. Check if Blob Exists

In [ ]:
# Check each candidate blob path
print(f"Checking blob existence for {len(blob_path_candidates)} candidate(s):\n")

existing_path = None
for path in blob_path_candidates:
    print(f"Checking: {path}")
    try:
        exists = azure_utils.blob_exists(path)
        if exists:
            print(f"  ✓ Found!")
            existing_path = path
            break
        else:
            print(f"  ✗ Not found")
    except Exception as e:
        print(f"  ✗ Error checking: {type(e).__name__}: {e}")

if existing_path:
    print(f"\n✓ SUCCESS: Found blob at {existing_path}")
else:
    print(f"\n✗ FAILED: None of the candidate paths exist")
    print(f"\nThis suggests either:")
    print(f"  1. Blobs were not uploaded to this prefix")
    print(f"  2. Azure authentication doesn't have read permissions")
    print(f"  3. Wrong container or storage account configured")

## 7. Download and Inspect Audio

In [ ]:
if existing_path:
    print(f"Downloading blob: {existing_path}\n")
    
    try:
        # Download to memory
        audio_bytes = azure_utils.download_blob_to_memory(existing_path)
        
        file_size_mb = len(audio_bytes) / (1024 * 1024)
        print(f"✓ Downloaded successfully!")
        print(f"  File size: {file_size_mb:.2f} MB ({len(audio_bytes):,} bytes)")
        
        # Check if it looks like valid audio data
        if len(audio_bytes) < 100:
            print(f"\n⚠️  WARNING: File is very small ({len(audio_bytes)} bytes) - may be corrupted")
        else:
            # Check file signature
            if existing_path.endswith('.mp3'):
                # MP3 files typically start with ID3 or FF FB/FF F3 (MPEG sync)
                header = audio_bytes[:4]
                if header[:3] == b'ID3' or header[:2] in [b'\xff\xfb', b'\xff\xf3']:
                    print(f"  ✓ File signature looks like valid MP3")
                else:
                    print(f"  ⚠️  File signature doesn't match MP3 (first 4 bytes: {header.hex()})")
            elif existing_path.endswith('.mp4'):
                # MP4 files have 'ftyp' at bytes 4-8
                header = audio_bytes[:12]
                if b'ftyp' in header:
                    print(f"  ✓ File signature looks like valid MP4")
                else:
                    print(f"  ⚠️  File signature doesn't match MP4 (first 12 bytes: {header.hex()})")
        
    except Exception as e:
        print(f"✗ Download failed: {e}")
        print(f"\nError type: {type(e).__name__}")
        import traceback
        print(f"\nFull traceback:\n{traceback.format_exc()}")
else:
    print("⊘ Skipping download - no blob found")

## 8. Load and Play Audio (Optional)

In [ ]:
if existing_path and 'audio_bytes' in locals():
    print("Loading audio with pydub...\n")
    
    try:
        import io
        from pydub import AudioSegment
        
        # Load audio from bytes
        audio_segment = AudioSegment.from_file(io.BytesIO(audio_bytes))
        
        print(f"✓ Audio loaded successfully!")
        print(f"  Duration: {len(audio_segment) / 1000:.1f} seconds ({len(audio_segment) / 60000:.1f} minutes)")
        print(f"  Sample rate: {audio_segment.frame_rate} Hz")
        print(f"  Channels: {audio_segment.channels} ({'mono' if audio_segment.channels == 1 else 'stereo'})")
        print(f"  Sample width: {audio_segment.sample_width} bytes")
        
        # Play first 10 seconds (if running locally with audio output)
        print(f"\n💡 To play audio, run: audio_segment[:10000].export('test.wav', format='wav')")
        
    except Exception as e:
        print(f"✗ Audio loading failed: {e}")
        print(f"\nError type: {type(e).__name__}")
else:
    print("⊘ Skipping audio playback - no audio downloaded")

## 9. Summary & Diagnostics

This notebook tests the full Azure blob authentication and access pipeline:

**Expected results if everything works:**
- ✓ Environment variables loaded
- ✓ Blob service client created
- ✓ Can list containers
- ✓ Target container exists
- ✓ Blob exists at expected path
- ✓ Can download blob
- ✓ File has valid audio signature
- ✓ Can load and inspect audio

**Common issues:**
1. **Env vars not loaded**: Check `credentials/creds.env` exists and has correct values
2. **Auth failed**: Check Azure credentials are valid and not expired
3. **Container not found**: Check `AZURE_STORAGE_CONTAINER` matches actual container name
4. **Blob not found**: Check `azure_blob_index` in parquet matches uploaded blobs
5. **Permission denied**: Check Azure credentials have Storage Blob Data Reader role

**Next steps if this succeeds:**
- Run `tutorial_infer_canary.ipynb` to test full inference pipeline
- This proves Azure access works, so any failures in inference are model-related